In [22]:
import pandas as pd
from scipy.io import arff
import os,sys

sys.path.append(os.path.abspath('../src'))
from src.data.load_data import load_arff_data

file_path = 'data\Supply Chain Management_train.arff'
df = load_arff_data(file_path)
target_features = [col for col in df.columns if col.startswith("MTLp") or col == "LBL"]
non_target_features = [col for col in df.columns if not (col.startswith("MTLp") or col == "LBL")]
print(f"total features = {len(non_target_features)}")
print(f"total target features = {len(target_features)}")
print(f"total rows = {len(df)}")
df.head()

total features = 280
total target features = 16
total rows = 8145


,timeunit,storageCost,interestRate,compidx0lt2,compidx0lt2l1,compidx0lt2l2,compidx0lt2l4,compidx0lt2l8,compidx1lt2,compidx2lt2,...,MTLp7,MTLp8,MTLp9,MTLp10,MTLp11,MTLp12,MTLp13,MTLp14,MTLp15,MTLp16
0,9.0,27.0,8.0,881.606,882.253,826.727,810.837,848.833,1468.412,815.832,...,2292.0,2181.0,1619.0,1849.0,1707.0,1932.0,2052.0,2409.0,2279.0,2138.0
1,10.0,27.0,8.0,869.455,881.606,882.253,802.480,770.544,1407.148,828.687,...,2276.0,2231.0,1649.0,1813.0,1696.0,1988.0,2140.0,2366.0,2307.0,2154.0
2,11.0,27.0,8.0,850.916,869.455,881.606,826.727,784.625,1424.312,837.975,...,2264.0,2203.0,1651.0,1739.0,1677.0,1991.0,2087.0,2295.0,2185.0,2131.0
3,12.0,27.0,8.0,862.066,850.916,869.455,882.253,792.403,1411.209,803.296,...,2279.0,2203.0,1627.0,1747.0,1669.0,1885.0,2072.0,2292.0,2187.0,2170.0
4,13.0,27.0,8.0,842.781,862.066,850.916,881.606,810.837,1428.685,813.118,...,2241.0,2198.0,1626.0,1740.0,1663.0,1874.0,2074.0,2257.0,2136.0,2164.0


In [35]:
from sklearn.model_selection import train_test_split

Y = df[target_features]
X = df[non_target_features]

# Split data menjadi train dan test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=19)


In [36]:
import sys
import os

# Tambahkan path ke folder src/models
sys.path.append(os.path.abspath('../src/models'))

# Pastikan path benar
print("Python Path:", sys.path)

# Import modul
from src.models.load_pca_model import load_pca_model, apply_pca_model

# Load PCA model
model_path = 'models\pca_model.pkl'
pca_model = load_pca_model(model_path)

# Transformasi data menggunakan PCA
if pca_model:
    X_train_pca = apply_pca_model(pca_model, X_train)
    X_test_pca = apply_pca_model(pca_model, X_test)
    

Python Path: ['c:\\Users\\fauza\\OneDrive\\Documents\\Job\\Skill Test\\Smartfren\\notebook', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310', '', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib', 'c:\\Users\\fauza\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin', 'C:\\Users\\fauza\\OneDrive\\Documents\\Job\\Skill Test\\src', 'C:\\Users\\fauza\\OneDrive\\Documents\\Job\\Skill Test\\src\\models', 'C:\\Users\\fauza\\OneDrive\\Documents\\Job\\Skill Test\\src\\models', 'C:\\Users\\fauza\\OneDrive\\Documents\\Jo

c:\Users\fauza\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(
c:\Users\fauza\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [37]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

import numpy as np
from sklearn.metrics import make_scorer

# Fungsi MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Scorer untuk cross-validation
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Initialize Random Forest Regressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=19)

# Wrap with MultiOutputRegressor for multitarget regression
multi_target_model = MultiOutputRegressor(random_forest)

# Cross-validation dengan MAPE
cv = KFold(n_splits=3, shuffle=True, random_state=19)
scores = cross_val_score(multi_target_model, X_train_pca, Y_train, cv=cv, scoring=mape_scorer)

# Print hasil
print("Cross-validation MAPE scores:", -scores)  # Negasi karena make_scorer menganggap error negatif lebih baik
print("Mean MAPE score:", -np.mean(scores))

Cross-validation MAPE scores: [3.90215184 3.86869087 3.8701998 ]
Mean MAPE score: 3.8803475049794502


In [53]:
# Latih model pada seluruh dataset
multi_target_model.fit(X_train_pca, Y_train)

# Prediksi dengan model
Y_pred = multi_target_model.predict(X_test_pca)

# Hitung MAPE
overall_mape = mean_absolute_percentage_error(Y_test, Y_pred)
print("Overall MAPE:", overall_mape)

Overall MAPE: 3.666268618408534


In [58]:
# import pickle

# with open('models/multi_target_model.pkl', 'wb') as f:
#     pickle.dump(multi_target_model, f)

import joblib
# joblib.dump(multi_target_model, 'models/multi_target_model_compressed.pkl', compress=('gzip', 3))

joblib.dump(multi_target_model, 'models/multi_target_model_compressed.pkl', compress=('lzma', 9))

['models/multi_target_model_compressed.pkl']